## Exotic Option Pricing with GPU-Accelerated MonteCarlo Simulations
Experiment for running large Monte-Carlo simulations for pricing an Asian Barrier Option. We will compare the runtimes of CPU-only runs as well as GPU-accelerated runs. For GPU acceleration, we will require a Nvidia Pascal or later GPU, along with the correct drivers installed. We'll also need the CuPY and numba libraries.

In [3]:
# Library imports:
import cupy
import numpy as np
import math
import time
import numba
from numba import cuda
from numba import njit
from numba import prange
#import cudf

cupy.cuda.set_allocator(None)

### Hardware specs:
We'll be comparing CPU single-threaded, CPU multi-threaded, and GPU performance runtimes. So first let's take a look at the machine's hardware specs.

In [4]:
# CPU thread count:
import multiprocessing
n_cpus = multiprocessing.cpu_count()
print('CPU threads:', n_cpus)

CPU threads: 20


In [2]:
# Verify GPU device is located:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


### Set parameters and run simulations

The specific option being simulated here is an Asian Barrier Down-and-Out. Stock price will be simulated based on the widely used Geometric Brownian Motion framework.
- Maturity (T): 1 year
- Spot (S) : 120
- Strike (K): 110
- Barrier (B): 100
- Volatility (sigma): 35.0 %
- Risk Free Rate (r): 5.0 %
- Stock Drift Rate (mu): 10.0 %

The price of the option, after simulation paths have been generated, is simply the discounted expected payoff.

In [5]:
# Define simulation parameters:
N_PATHS = 8000000
N_STEPS = 365

T = 1.0
K = 110.0
B = 100.0
S0 = 120.0
sigma = 0.35
mu = 0.1
r = 0.05

In [6]:
# We use CuPy to generate Gaussian random numbers in the GPU and allocate an array to store the prices at maturity.
randoms_gpu = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)
randoms_cpu = cupy.asnumpy(randoms_gpu)

# Output array:
output = np.zeros(N_PATHS, dtype=np.float32)

##### Single Thread CPU
The single thread CPU code for the simulation has two nested for-loops. The outer loop iterates each path (the possible paths that the underlying asset might take) while the inner loop computes the underlying asset price for that path by iterating through the 365 days of the path. Notice that for this pricing exercise we're limiting ourselves to daily price changes, rather than intra-day changes.

Notice the `break` in the inner loop - this is due to the Down-and-Out barrier being breached, and hence the option being voided.

To speed things up, we're using Numba's @jit decorator, hence the code compiles into machine code at runtime.

In [7]:
# Single-threaded run. Using `fastmath` helps to improve runtime as it relaxes some floating point compute rigor requirements.
@njit(fastmath=True)
def cpu_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS):
    tmp1 = mu*T/N_STEPS
    tmp2 = math.exp(-r*T)
    tmp3 = math.sqrt(T/N_STEPS)

    for i in range(N_PATHS):
        s_curr = S0
        running_average = 0.0
        for n in range(N_STEPS):
            s_curr += tmp1 * s_curr + sigma*s_curr*tmp3*d_normals[i + n * N_PATHS]
            running_average = running_average + 1.0/(n + 1.0) * (s_curr - running_average)
            if running_average <= B:
                break

        payoff = running_average - K if running_average>K else 0
        d_s[i] = tmp2 * payoff

In [8]:
# Run Monte Carlo:
t0 = time.time()
cpu_barrier_option(output, np.float32(T), np.float32(K),
                    np.float32(B), np.float32(S0),
                    np.float32(sigma), np.float32(mu),
                    np.float32(r), randoms_cpu, N_STEPS, N_PATHS)
print('Single-threaded CPU runtime:', time.time() - t0)

exp_payoff = output.mean()
print('Simulated expected payoff:', exp_payoff)

Single-threaded CPU runtime: 11.732485294342041
Simulated expected payoff: 18.703484


##### Multi-Thread CPU
We can parallelize the outer loop (which are all independent) by calling the outer loop with `prange` instead of `range`. Also note the `parallel = True` adjustment in the decorator.

In [9]:
# Multi-threaded run:
@njit(fastmath=True, parallel=True)
def cpu_multicore_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS):
    tmp1 = mu*T/N_STEPS
    tmp2 = math.exp(-r*T)
    tmp3 = math.sqrt(T/N_STEPS)

    for i in prange(N_PATHS):
        s_curr = S0
        running_average = 0.0
        for n in range(N_STEPS):
            s_curr += tmp1 * s_curr + sigma*s_curr*tmp3*d_normals[i + n * N_PATHS]
            running_average = running_average + 1.0/(n + 1.0) * (s_curr - running_average)
            if running_average <= B:
                break

        payoff = running_average - K if running_average>K else 0
        d_s[i] = tmp2 * payoff

In [10]:
# Run Monte Carlo:
t0 = time.time()
cpu_multicore_barrier_option(output, np.float32(T), np.float32(K),
                            np.float32(B), np.float32(S0),
                            np.float32(sigma), np.float32(mu),
                            np.float32(r), randoms_cpu, N_STEPS, N_PATHS)
print('Multi-threaded CPU runtime:', time.time() - t0)

exp_payoff = output.mean()
print('Simulated expected payoff:', exp_payoff)

Multi-threaded CPU runtime: 1.009613275527954
Simulated expected payoff: 18.703484


##### GPU with Numba
It's also easy to parallelize the calculations to GPU via Numba using the `Numba.cuda.jit` decorator. The code below is very similar to the CPU multiple core code except that we parallelize the outer loop on the GPU.

In [11]:
# GPU run. Notice the `@cuda.jit` decorator, which is imported from Numba.
@cuda.jit
def numba_gpu_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS):
    # Here, ii is the overall thread index. Note how they're used later in the for-loop.
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x

    tmp1 = mu*T/N_STEPS
    tmp2 = math.exp(-r*T)
    tmp3 = math.sqrt(T/N_STEPS)

    for i in range(ii, N_PATHS, stride):
        s_curr = S0
        running_average = 0.0
        for n in range(N_STEPS):
            s_curr += tmp1 * s_curr + sigma*s_curr*tmp3*d_normals[i + n * N_PATHS]
            running_average += (s_curr - running_average) / (n + 1.0)
            if running_average <= B:
                break

        payoff = running_average - K if running_average>K else 0
        d_s[i] = tmp2 * payoff

In [12]:
# GPU run:
number_of_threads = 256
number_of_blocks = (N_PATHS-1) // number_of_threads + 1
output = cupy.zeros(N_PATHS, dtype=cupy.float32)

t0 = time.time()
numba_gpu_barrier_option[(number_of_blocks,), (number_of_threads,)](output, np.float32(T), np.float32(K), 
                    np.float32(B), np.float32(S0), 
                    np.float32(sigma), np.float32(mu), 
                    np.float32(r), randoms_gpu, N_STEPS, N_PATHS)
cuda.synchronize()
print('GPU runtime:', time.time() - t0)

exp_payoff = output.mean()
print('Simulated expected payoff:', exp_payoff)

GPU runtime: 0.6308526992797852
Simulated expected payoff: 18.70347


#### Clear memory

In [13]:
del randoms_gpu
del randoms_cpu
del output